# Prompt Evolver

This notebook contains the local work setup and example run for the Prompt Evolver.


## 1. Work setup (use .venv)

Run these commands in a terminal (notebook kernel cannot create the venv itself):

```bash
python -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
```

Then select the `.venv` interpreter as the notebook kernel.


## 2. Constants (file paths + prompts)

Adjust these paths to point at your own datasets and prompt templates.


In [1]:
from pathlib import Path
import sys

def find_project_root(start: Path) -> Path:
    for parent in [start, *start.parents]:
        if (parent / "src" / "prompt_evolver").exists():
            return parent
    raise RuntimeError("Could not locate project root containing src/prompt_evolver")

project_root = find_project_root(Path.cwd())
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

PROMPTS_PATH = project_root / "data" / "prompts.csv"
TEXTS_PATH = project_root / "data" / "texts.csv"
TASKS_PATH = project_root / "data" / "tasks.csv"
RESULTS_PATH = project_root / "data" / "results.csv"

EXAMPLE_PROMPTS_PATH = project_root / "data" / "example.prompts.csv"
EXAMPLE_TEXTS_PATH = project_root / "data" / "example.texts.csv"
EXAMPLE_TASKS_PATH = project_root / "data" / "example.tasks.csv"
EXAMPLE_RESULTS_PATH = project_root / "data" / "example.results.csv"

CONFIG_PATH = project_root / "configs" / "config.yaml"
EXECUTION_PROMPT_PATH = project_root / "configs" / "prompts" / "prompt.execute.md"
IMPROVEMENT_PROMPT_PATH = project_root / "configs" / "prompts" / "prompt.improve.md"
EVALUATION_PROMPT_PATH = project_root / "configs" / "prompts" / "prompt.evaluation.md"

MAX_GENERATIONS = 3


## 3. Example run

This executes the pipeline using the example CSVs and prompt templates.


### Kernel note

If you changed code in `src/`, restart the kernel and run the notebook cells from the top.


In [2]:
from prompt_evolver.config import load_config
from prompt_evolver.pipeline import run_pipeline

config = load_config(CONFIG_PATH)
execution_template = EXECUTION_PROMPT_PATH.read_text(encoding="utf-8")
improvement_template = IMPROVEMENT_PROMPT_PATH.read_text(encoding="utf-8")
evaluation_template = EVALUATION_PROMPT_PATH.read_text(encoding="utf-8")

results_df = run_pipeline(
    EXAMPLE_PROMPTS_PATH,
    EXAMPLE_TEXTS_PATH,
    EXAMPLE_TASKS_PATH,
    EXAMPLE_RESULTS_PATH,
    config=config,
    execution_prompt_template=execution_template,
    improvement_prompt_template=improvement_template,
    evaluation_prompt_template=evaluation_template,
    max_generations=MAX_GENERATIONS,
)
results_df.head()

2026-01-22 23:44:24,003 [INFO] Start pipeline prompts=/Users/bru1t/Documents/Development/projects/prompt-evolver/data/example.prompts.csv texts=/Users/bru1t/Documents/Development/projects/prompt-evolver/data/example.texts.csv tasks=/Users/bru1t/Documents/Development/projects/prompt-evolver/data/example.tasks.csv output=/Users/bru1t/Documents/Development/projects/prompt-evolver/data/example.results.csv
2026-01-22 23:44:24,016 [INFO] Start task id_task=task_001 id_prompt=prompt_001 id_text=text_001 type=Writing
2026-01-22 23:44:45,370 [WARNING] Eval failed task=task_001 score=0.75 issues=["The phrasing 'optimizes prompt refinement' is less concise than the expected output (e.g., 'helps teams refine prompts').", "Missing explicit mention of 'comparing model outputs against expected answers' in the first bullet.", "The second bullet lacks clarity on 'token usage and guardrails' implementation specifics beyond general cost efficiency.", "The third bullet is slightly redundant with phrasing 

,id_task,id_text,id_prompt,prompt_original,prompt_improved,tokens_original,tokens_improved,tokens_delta,iterations_used,output_original,...,output_tokens_original,output_tokens_improved,evaluation_original,evaluation_improved,model_task,model_improve,model_eval,leakage_flag,sanity_check_details,failure_reason
0,task_001,text_001,prompt_001,Summarize the text in 3 bullet points.,Summarize the text in 3 bullet points.,8,8,0,2,- **Optimizes prompt refinement** by comparing...,...,86,86,"{""pass"": false, ""score"": 0.75, ""issues"": [""The...","{""pass"": false, ""score"": 0.75, ""issues"": [""The...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,too_large_increase,no_improvement
1,task_002,text_002,prompt_002,Extract the user's name and status in JSON.,Extract `{name}` and `{status}` from `{text}` ...,11,32,21,1,"```json\n{\n ""name"": ""Alice Nguyen"",\n ""stat...",...,39,25,"{""pass"": false, ""score"": 0.3, ""issues"": [""Miss...","{""pass"": true, ""score"": 1.0, ""issues"": [], ""su...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,,
2,task_003,text_003,prompt_003,Rewrite the text to be clearer and shorter.,Rewrite the text to be clearer and shorter.,9,9,0,2,The product delivers significantly faster perf...,...,70,70,"{""pass"": false, ""score"": 0.65, ""issues"": [""The...","{""pass"": false, ""score"": 0.65, ""issues"": [""The...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,too_large_increase,sanity_check:too_large_increase
3,task_004,text_004,prompt_004,Compare the two products in the text and list ...,Compare the two products in the text and list ...,12,12,0,2,1. **Warranty Duration**: Product A has a **2-...,...,105,105,"{""pass"": false, ""score"": 0.6, ""issues"": [""Miss...","{""pass"": false, ""score"": 0.6, ""issues"": [""Miss...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,too_large_increase,no_improvement
4,task_005,text_005,prompt_005,Evaluate the text for policy compliance and an...,Evaluate the text for policy compliance and an...,12,12,0,1,pass,...,1,1,"{""pass"": true, ""score"": 1.0, ""issues"": [], ""su...","{""pass"": true, ""score"": 1.0, ""issues"": [], ""su...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,,


## 4. Inspect output


In [3]:
import pandas as pd

pd.read_csv(EXAMPLE_RESULTS_PATH).head(20)


,id_task,id_text,id_prompt,prompt_original,prompt_improved,tokens_original,tokens_improved,tokens_delta,iterations_used,output_original,...,output_tokens_original,output_tokens_improved,evaluation_original,evaluation_improved,model_task,model_improve,model_eval,leakage_flag,sanity_check_details,failure_reason
0,task_001,text_001,prompt_001,Summarize the text in 3 bullet points.,Summarize the text in 3 bullet points.,8,8,0,1,- **Optimizes prompt refinement** by comparing...,...,85,85,"{""pass"": true, ""score"": 0.95, ""issues"": [""Mino...","{""pass"": true, ""score"": 0.95, ""issues"": [""Mino...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,NaN,NaN
1,task_002,text_002,prompt_002,Extract the user's name and status in JSON.,json\nExtract `{name}` and `{status}` from {te...,11,29,18,1,"```json\n{\n ""name"": ""Alice Nguyen"",\n ""stat...",...,39,25,"{""pass"": false, ""score"": 0.5, ""issues"": [""Incl...","{""pass"": true, ""score"": 1.0, ""issues"": [], ""su...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,NaN,NaN
2,task_003,text_003,prompt_003,Rewrite the text to be clearer and shorter.,Rewrite the text to be clearer and shorter.,9,9,0,2,**Revised version:**\n\nThe draft is overly wo...,...,90,90,"{""pass"": false, ""score"": 0.75, ""issues"": [""The...","{""pass"": false, ""score"": 0.75, ""issues"": [""The...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,NaN,no_improvement
3,task_004,text_004,prompt_004,Compare the two products in the text and list ...,Compare the two products in the text and list ...,12,12,0,2,1. **Warranty Duration**: Product A has a **tw...,...,108,108,"{""pass"": false, ""score"": 0.3, ""issues"": [""Miss...","{""pass"": false, ""score"": 0.3, ""issues"": [""Miss...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,too_large_increase,sanity_check:too_large_increase
4,task_005,text_005,prompt_005,Evaluate the text for policy compliance and an...,"**""Is {text} policy-compliant? Return pass/fai...",12,19,7,1,Pass,...,1,1,"{""pass"": true, ""score"": 1.0, ""issues"": [], ""su...","{""pass"": true, ""score"": 1.0, ""issues"": [], ""su...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,NaN,NaN
5,task_006,text_006,prompt_006,"Extract key metrics as JSON with keys revenue,...","Extract key metrics as JSON with keys revenue,...",13,13,0,1,"```json\n{\n ""revenue"": ""$1.2M"",\n ""users"": ...",...,38,38,"{""pass"": true, ""score"": 1.0, ""issues"": [], ""su...","{""pass"": true, ""score"": 1.0, ""issues"": [], ""su...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,NaN,NaN
6,task_007,text_007,prompt_007,Research summary: provide 2 takeaways and 1 op...,Research summary: provide 2 takeaways and 1 op...,11,11,0,2,**Takeaways:**\n1. The drug shows **short-term...,...,132,132,"{""pass"": false, ""score"": 0.75, ""issues"": [""The...","{""pass"": false, ""score"": 0.75, ""issues"": [""The...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,too_large_increase,no_improvement
7,task_008,text_008,prompt_008,Provide operational steps to execute the reque...,Provide operational steps to execute the reque...,9,9,0,3,1. **Plan Rollout Window**\n - Schedule depl...,...,650,650,"{""pass"": false, ""score"": 0.0, ""issues"": [""inva...","{""pass"": false, ""score"": 0.0, ""issues"": [""inva...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,NaN,max_iterations
8,task_009,text_009,prompt_009,Extract action items from the text.,Extract action items from the text.,7,7,0,2,- **Action Items:**\n - Marta to draft the be...,...,55,55,"{""pass"": false, ""score"": 0.5, ""issues"": [""Miss...","{""pass"": false, ""score"": 0.5, ""issues"": [""Miss...",mistralai/ministral-3-3b,mistralai/ministral-3-3b,mistralai/ministral-3-3b,no,NaN,no_improvement
9,task_010,text_010,prompt_010,Redact perso

## 5. Run unit tests

Run this in a terminal with the `.venv` active:

```bash
pytest
```
